In [ ]:
# Imports
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import os
from pathlib import Path
import string
import pandas as pd
import numpy as np
import seaborn as sns
import panel as pn
from panel.interact import interact, interactive, fixed, interact_manual
from panel import widgets
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pn.extension("plotly")
from pytrends.request import TrendReq

In [ ]:
# Load .env
load_dotenv()

In [ ]:
# Set Alpaca API key and Secret key
alpaca_key = os.getenv("ALPACA_API_KEY")
alpaca_secret = os.getenv("ALPACA_API_SECRET")

# Create the Alpaca API object
alpaca = tradeapi.REST(alpaca_key, alpaca_secret, api_version = "vs")
timeframe = "1D"
start = pd.Timestamp("2016-05-26", tz = "US/Pacific").isoformat()
end = pd.Timestamp("2021-06-6", tz = "US/Pacific").isoformat()

In [ ]:
# Declares Global Variables
pytrend = TrendReq()

sectors = [
    'Communications',
    'Consumer Discretionary',
    'Consumer Staples',
    'Energy',
    'Financial',
    'Healthcare',
    'Industrial',
    'Information Technology',
    'Materials',
    'Real Estate',
    'Utilities'
    ]

beta = ['Min', 'Max', 'Median', 'Mutual Fund']

z_field = ['Close', 'Volume']

sector_tickers = {
    'Communications':
        {'Min': 'VZ', 'Max': 'LYV', 'Median': 'TMUS', 'Mutual Fund': 'VOX'},
    'Consumer Discretionary':
        {'Min': 'NVR', 'Max': 'F', 'Median': 'HLT', 'Mutual Fund': 'VCR'},
    'Consumer Staples':
        {'Min': 'CLX', 'Max': 'SYY', 'Median': 'PM', 'Mutual Fund': 'VDC'},
    'Energy':
        {'Min': 'COG', 'Max': 'OXY', 'Median': 'SLB', 'Mutual Fund': 'VDE'},
    'Financial':
        {'Min': 'CBOE', 'Max': 'LNC', 'Median': 'BAC', 'Mutual Fund': 'VFH'},
    'Healthcare':
        {'Min': 'DGX', 'Max': 'ALGN', 'Median': 'CAH', 'Mutual Fund': 'VHT'},
    'Industrial':
        {'Min': 'DGX', 'Max': 'TDG', 'Median': 'DE', 'Mutual Fund': 'VIS'},
    'Information Technology':
        {'Min': 'ORCL', 'Max': 'ENPH', 'Median': 'NTAP', 'Mutual Fund': 'VGT'},
    'Materials':
        {'Min': 'NEM', 'Max': 'FCX', 'Median': 'AVY', 'Mutual Fund': 'VAW'},
    'Real Estate':
        {'Min': 'PSA', 'Max': 'SPG', 'Median': 'UDR', 'Mutual Fund': 'VNQ'},
    'Utilities':
        {'Min': 'ED', 'Max': 'AES', 'Median': 'SRE', 'Mutual Fund': 'VPU'}
}

member_picks = {
'Boomer': ['VDC', 'VNQ', 'VOX', 'VAW'],
'Stonks': ['GME', 'AMC', 'PSLV', 'BB'],
'Pro Gamer': ['AAPL', 'TSLA', 'AMC', 'WMT'],
'Real American': ['LMT', 'TAP', 'PM', 'HAL']
}

In [ ]:
# Panel Text
group_members = '''
## Group 4
* Mouhamadou Lmine Diop aka 🎮 Pro Gamer 🕹️
* Josh Ferguson aka 🦅 Proud Patriot 🇺🇸
* Ben McCright aka 👵 Boomer 👴
* Cole Wood aka 💎 Diamond Hands 👐
'''

box_blurb = '''
## Risk of each Sector and S & P 500 index, select the Beta:'''

heat_blurb = '''
## Correlation of each Sector and the S & P 500 index, select the Beta:'''

trend_blurb = '''
## Closing price and Goolge search interest: select Sector and Beta:'''

candlestick_blurb = '''
## Candlestick chart, select Sector and Beta:'''

spy_blurb = '''
## Compares closing price to SPY fund, select Sector and Beta:'''

tt_blurb = '''
## Go back in time to 2017 and see where your investments would be now, enter 3 Stocks and Amounts:
### (Warning: Does not reflect stock splits)'''

sharpe_blurb = '''
## Sharpe ratio of 4 stocks chosen by group members, select Portfolio:'''

In [ ]:
# Generates Correlation Heatmap of Sector Mutual Funds & Index
def df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}

@interact(Beta = beta)
def heatmap(Beta):
    df = pd.DataFrame()
    sp_file = Path('../Data/SP500.csv')
    sp_df = pd.read_csv(sp_file, infer_datetime_format=True, parse_dates=True, index_col='Date')
    df['SP500'] = sp_df['Close']
    for k, v in sector_tickers.items():
        ticker = sector_tickers[k][Beta]
        file = Path('../Data/{}.csv'.format(ticker))
        ticker_df = pd.read_csv(file, infer_datetime_format=True, parse_dates=True, index_col='Date')
        df[k] = ticker_df['Close']
    df = df.pct_change()
    df.dropna(inplace = True)
    corr = df.corr()
    fig = go.Figure(data=go.Heatmap(
        df_to_plotly(corr),
        colorscale='blues'))
    fig.update_layout(title = 'Heatmap',width=1000, height=500)
    return fig

In [ ]:
# Generates Candlestick Chart of Sector Ticker
@interact(Sector = sectors, Beta = beta)
def candlestick(Sector, Beta):
    ticker = sector_tickers[Sector][Beta]
    file = Path('../Data/{}.csv'.format(ticker))
    df = pd.read_csv(file, infer_datetime_format=True, parse_dates=True, index_col='Date')
    fig = go.Figure(data=[go.Candlestick(
        x = df.index,
        open = df['Open'],
        high = df['High'],
        low = df['Low'],
        close = df['Close']
    )])
    fig.update_xaxes(title_text = 'Date')
    fig.update_yaxes(title_text = 'Price (USD)')
    fig.update_layout(title = ticker, width=1000, height=500)
    return fig

In [ ]:
# Generates Comparison Line Graph of Sector Ticker & SPY
@interact(Sector = sectors, Beta = beta)
def v_spy(Sector, Beta):
    ticker = sector_tickers[Sector][Beta]
    file = Path('../Data/{}.csv'.format(ticker))
    df = pd.read_csv(file, infer_datetime_format=True, parse_dates=True, index_col='Date')
    spy = pd.read_csv('../Data/SPY.csv', infer_datetime_format=True, parse_dates=True, index_col='Date')
    fig = make_subplots()
    trace1 = go.Scatter(
        x = df.index,
        y = df['Close'],
        mode = 'lines',
        name = ticker)
    trace2 = go.Scatter(
        x = spy.index,
        y = spy['Close'],
        mode = 'lines',
        name = 'SPY')    
    fig.add_trace(trace1)
    fig.add_trace(trace2)
    fig.update_xaxes(title_text = 'Date')
    fig.update_yaxes(
        title_text = 'Closing Price (USD)')
    fig.update_layout(title = ticker + " versus SPY", width=1000, height=500)
    return fig